# Web Scraping Extension

As an extension we'll be scraping Wikipedia to determine whether there is a link between hours of sunshine in a country and the rate of depression.

Data about depression can be found here: https://en.wikipedia.org/wiki/Epidemiology_of_depression

Data bout sun hours: https://en.wikipedia.org/wiki/List_of_cities_by_sunshine_duration

We will proceed as follows:
- Scrape 1st link to get DALY rate per country
- Scrape 2nd link to get sun hours per city
- Aggregate data to get sun hours per country
- Filter data to match every country to its DALY rate and sun rate
- Visualise data
- Preprocess & model

A few imports and constants we'll be needing:

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from requests import get
import numpy as np
import pandas as pd

URL_DALY = 'https://en.wikipedia.org/wiki/Epidemiology_of_depression'
URL_SUN = 'https://en.wikipedia.org/wiki/List_of_cities_by_sunshine_duration'
FILE = './data-extension.csv'

## Scraping DALY rate per country

Our goal here is to get a dictionary containing every country mapped to its DALY rate. We start by getting the URL and creating a soup.

In [ ]:
resp = get(URL_DALY)
soup = BeautifulSoup(resp.text, 'html.parser')

Now extract the rows of the table. We can safely ignore the first row as it contains the header of the table.

In [ ]:
rows = soup.select('table > tbody > tr')[1:]

Given the rows now extract the data into a dictionary which maps every country to its DALY rate. The `string.strip()` and `string.replace(..)` functions are useful to format the data as we want it.

In [ ]:
data_daly = {}
for row in rows:
    vals = row.select('td')[1:]
    
    # Make sure values exist
    if vals == []:
        continue
    
    country = vals[0].text.strip()
    daly = float(vals[1].text.replace(',', ''))
    
    data_daly[country] = daly

## Scraping sun hours per city
We now need to scrape for sun hours. This is a bit more tricky since the data is spread across multiple tables and multiple cities. We start off by creating a mapping from country to a list of sun hours in all of its cities. For instance, say Botswana has 3 cities listed, we will have the following mapping: `'Botswana': [3330.0, 3371.0, 3579.0]`. We'll then aggregate all the data to get a single averaged value per country.

We start off by collecting all the tables of the page into a single array with all the rows (i.e. all the `td` tags). Note that the first two tables of the page do not contain any data we want and can be safely ignored.

In [ ]:
resp = get(URL_SUN)
soup = BeautifulSoup(resp.text, 'html.parser')

tables = soup.select('table > tbody')[2:]

Now aggregate all the rows from the tables into a single array of rows.

In [ ]:
rows_sun = []
for table in tables:
    rows = table.select('tr')[1:]
    rows_sun += rows

We can now create a mapping from country to an array of sun hours per city (as mentioned above). Note that the sunhours per year can be found in column 14 (starting at 0). Again, it can be useful to use the `string.replace(..)` function to format number data.

In [ ]:
sun_cities = {}
for row in rows_sun:
    vals = row.select('td')
    country = vals[0].text.replace('\n', '')
    sun = float(vals[14].text.replace(',', ''))
    
    # If we already have data for the country, we append to the array
    if country in sun_cities:
        sun_cities[country].append(sun)
    else: # Oterwise, we create a new array with the data
        sun_cities[country] = [sun]

## Aggregating data to hours per country

We can now aggregate the data from sun hours per city to sun hours per country by just taking the mean of every city for every country. This will give us our final mapping from country to sun hours.

In [ ]:
data_sun = {}
for country, cities in sun_cities.items():
    data_sun[country] = sum(cities) / len(cities)

## Filter data to match every country to its DALY rate and sun rate
Now that we have two mappings `'country': daly` and `'country': 'sun'`, we wich to aggregate the data into the following format `[[country, daly, sun], [country, daly, sun], ..]`. This will make it easier to work with the data after. Note that there may be cases of missing data for some countries. In that case we should just skip those and keep countries which both have a DALY rate and a sunhours entry. We can therefore start by getting the list of countries that have data for both:

In [ ]:
# Get list of countries that both have a DALY rate and sunhours entry
countries_daly = set(data_daly.keys())
countries_sun = set(data_sun.keys())
countries = countries_daly & countries_sun # Use set intersection

Now create the dataset following the format described above:

In [ ]:
data = []
for country in countries:
    data.append([country, data_daly[country], data_sun[country]])

We can now put our data into a pandas dataframe and save it for reuse.

In [ ]:
data = pd.DataFrame(data, columns=['country', 'daly', 'sunhours'])
# Save data to file at path FILE
data.to_csv(FILE, sep=',', index=False)

We can now simply load our data from a file instead of web scraping again.

In [ ]:
data = pd.read_csv(FILE)

## Visualise data
Now that we have our fancy data we can use matplotlib to plot a few graphs. We can start off by plotting daly rate for sunhours. We use `plt.scatter(..)` to display points.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.xlabel('sunhours')
plt.ylabel('daly')
plt.scatter(data['sunhours'], data['daly'])

## Preprocess & model data
Now that we have our data, we can normalise it and train a model on it. We start off by importing some libraries and normalising the data to have it fit in a `0-1` range. For reference, the min-max normalisation formula is $X_{normalised}=\frac{X-min(X)}{max(X)-min(X)}$. Note that it might be useful to drop the country name from the data and just keep the daly and sun rates.

In [ ]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

data = data[['daly', 'sunhours']]
data = (data - data.min()) / (data.max() - data.min())

We now split the data into `X` and `y`, respectively for input features and output feature. We have a single input which is the sunhours and our output feature is the daly rate we want to predict. We also need to convert our data from a pandas dataframe to a numpy array using the `to_numpy()` method.

In [ ]:
X = data['sunhours'].to_numpy().reshape(-1, 1)
y = data['daly'].to_numpy().reshape(-1, 1)

Using sklearn's `train_test_split` function we now spearate our data into a training and test set to end up with `X_train`, `y_train`, `X_test`, `y_test`:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.9,
    test_size=0.1,
    random_state=0
)

And finally let's fit a simple linear regression model to it:

In [ ]:
reg = LinearRegression()
reg.fit(X=X_train, y=y_train)
reg.score(X=X_test, y=y_test)

_William Profit (williamprofit.com) on behalf of ICDSS (icdss.uk)_